# Project: Wrangle and Analyze Data from @WeRateDogs
## Introduction

For this project we are required to apply our data wrangling skills to gather, assess and clean data coming from twitter account [WeRateDogs](https://twitter.com/dog_rates/) and then generate some insights and visualizations.

So lets start our project :)

## Gather

Data Wrangling is an iterative proccess but just to have some structure I will start by working on gathering the necessary data and assing it to an individual dataframe. This will come from 3 sources:

- `twitter-archive-enhanced.csv` which I have already uploaded to this notebook
- `image_predictions.tsv` I need to scrap it from the web using *Requests*
- `tweet_json.txt` built from scratch using *tweepy* and twitter's API keys

### `twitter-archive-enhanced.csv`
This is the easiest one since I only have to read it using pandas [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.

*Note: For simplicity I will be importing the required libraries at the first moment I need them.*

In [1]:
# Importing pandas to the notebook
import pandas as pd

In [2]:
# Reading from the csv and assign to a dataframe
csv_df = pd.read_csv('twitter-archive-enhanced.csv')

# Checking df
csv_df.sample()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1817,676603393314578432,NaN,NaN,2015-12-15 03:23:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Godzilla pupper. He had a ruff childho...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/676603393...,9,10,Godzilla,None,None,pupper,None


### `image_predictions.tsv`
For this one, altough I could manually download it, I am required to apply my knowledge of [`requests`](https://pypi.org/project/requests/) and gather the file directly from an URL into a .tsv file using [`open()`](https://docs.python.org/3/library/functions.html#open) and finally assign it to a dataframe called `predictions_df`.

In [3]:
# Importing requests and os library
import requests
import os

In [4]:
# Assign URL provided by Udacity to a variable
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

# Get the website data into a variable
predictions = requests.get(url)

In [5]:
# Checking encoding to ensure is utf-8
predictions.encoding

'utf-8'

In [6]:
# Save website data to .tsv file
with open('image_predictions.tsv', mode='w') as file:
    file.write(predictions.text)

# Assign tsv file to dataframe
predictions_df = pd.read_csv('image_predictions.tsv', sep='\t')

# Check df
predictions_df.sample()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
379,673270968295534593,https://pbs.twimg.com/media/CVfwXuWWIAAqnoi.jpg,1,Shih-Tzu,0.610453,True,Maltese_dog,0.166815,True,Old_English_sheepdog,0.132015,True


### `tweet_json.txt`
Once I got access to Twitter's API I was able to extract additional information into a JSON file, now let's read the file:

In [7]:
# Reading from the json file and assign to a dataframe
json_df = pd.read_json('tweet_json.txt', lines=True)

# Checking df
json_df.sample()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
1201,NaN,NaN,2016-03-27 17:25:54,"[0, 137]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 714141403652231168, 'id_str'...",4241,False,"I know we only rate dogs, but since it's Easte...",NaN,...,NaN,NaN,NaN,NaN,1362,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


## Assess
Now that we have all of our data we need to identify Quality and Tidiness issues in each of our data frames, I will start by exploring them visually here in the notebook to get a glympse of some of the problems we need to fix.

But first, since I don't like how dataframes are viewed here in jupyter notebook, I will change some display options to my prefered values.

In [8]:
# Add CSS style to dataframe headers and data to avoid text wrapping when displaying df 
# (https://stackoverflow.com/a/57828617), also added borders and left aligment.

In [9]:
%%html
<style>
.dataframe th {
    white-space: nowrap;
    border: 1px solid black;
    text-align-last: left;
}
.dataframe td {
    white-space: nowrap;
    border: 1px solid black;
    text-align: left;
}
</style>

In [10]:
# Change pandas options to display almost all characters of every cell
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html
pd.set_option('display.max_seq_items', None)
pd.set_option('display.max_colwidth', 99999)

### Assessing `csv_df`

In [11]:
# View original csv file 'twitter-archive-enhanced.csv' dataframe
csv_df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
498,813130366689148928,8.131273e+17,4.196984e+09,2016-12-25 21:12:41 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",I've been informed by multiple sources that this is actually a dog elf who's tired from helping Santa all night. Pupgraded to 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
559,803276597545603072,NaN,NaN,2016-11-28 16:37:19 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Winston. His selfie game is legendary. Will steal your girl with a single snap. 11/10 handsome as h*ck https://t.co/jxQhxoPsgL,NaN,NaN,NaN,https://twitter.com/dog_rates/status/803276597545603072/photo/1,11,10,Winston,None,None,None,None
928,754747087846248448,NaN,NaN,2016-07-17 18:38:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Keith. He's pursuing a more 2D lifestyle. Idiosyncratic af. 12/10 follow your dreams Keith https://t.co/G9ufksBMlU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/754747087846248448/photo/1,12,10,Keith,None,None,None,None
1155,724983749226668032,NaN,NaN,2016-04-26 15:29:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Fred-Rick. He dabbles in parkour. The elevation gives him power. 12/10 hopefully visiting a mailbox near you https://t.co/qFqLtudIiD,NaN,NaN,NaN,https://twitter.com/dog_rates/status/724983749226668032/photo/1,12,10,Fred,None,None,None,None
1475,693642232151285760,NaN,NaN,2016-01-31 03:49:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Freddery. He's a Westminster Toblerone. Seems to enjoy car rides. 9/10 would pat on the head approvingly https://t.co/6BS9XEip9a,NaN,NaN,NaN,https://twitter.com/dog_rates/status/693642232151285760/photo/1,9,10,Freddery,None,None,None,None


In [12]:
# Check null values
csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [13]:
# Check RTs to understand better what type of post they represent, for example the page retweeting itself
csv_df[csv_df.text.str.startswith('RT')].sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
475,816062466425819140,NaN,NaN,2017-01-02 23:23:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: Meet Jack. He's one of the rare doggos that doesn't mind baths. 11/10 click the link to see how you can help Jack!\n\nhttps://…,8.159907e+17,4.196984e+09,2017-01-02 18:38:42 +0000,"https://www.gofundme.com/surgeryforjacktheminpin,https://twitter.com/dog_rates/status/815990720817401858/photo/1",11,10,Jack,None,None,None,None
911,757597904299253760,NaN,NaN,2016-07-25 15:26:30 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @jon_hill987: @dog_rates There is a cunningly disguised pupper here mate! 11/10 at least. https://t.co/7boff8zojZ,7.575971e+17,2.804798e+08,2016-07-25 15:23:28 +0000,"https://twitter.com/jon_hill987/status/757597141099548672/photo/1,https://twitter.com/jon_hill987/status/757597141099548672/photo/1",11,10,None,None,None,pupper,None
431,820837357901512704,NaN,NaN,2017-01-16 03:37:31 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Finn. He's wondering if you come here often. Fr*ckin flirtatious af. 12/10 would give number to https://t.co/ii5eNX5…,8.192277e+17,4.196984e+09,2017-01-11 17:01:16 +0000,https://twitter.com/dog_rates/status/819227688460238848/photo/1,12,10,Finn,None,None,None,None
629,794355576146903043,NaN,NaN,2016-11-04 01:48:22 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is Butter. She can have whatever she wants forever. 12/10 would hug softly https://t.co/x5gXRS1abq,7.887659e+17,4.196984e+09,2016-10-19 15:37:03 +0000,"https://twitter.com/dog_rates/status/788765914992902144/photo/1,https://twitter.com/dog_rates/status/788765914992902144/photo/1",12,10,Butter,None,None,None,None
599,798682547630837760,NaN,NaN,2016-11-16 00:22:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: Here we see a rare pouched pupper. Ample storage space. Looks alert. Jumps at random. Kicked open that door. 8/10 https://t.…,6.769365e+17,4.196984e+09,2015-12-16 01:27:03 +0000,https://twitter.com/dog_rates/status/676936541936185344/photo/1,8,10,None,None,None,pupper,None


In [14]:
# Check records without photo, seems like they are videos instead.
csv_df[csv_df.expanded_urls.str.contains('photo')==False].shape[0]

220

In [15]:
# Checking for duplicated tweet ids
csv_df[csv_df['tweet_id'].duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [16]:
# Checking statistics for numeric columns
csv_df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [17]:
# Checking name values
csv_df.name.value_counts()

None            745
a                55
Charlie          12
Oliver           11
Cooper           11
Lucy             11
Penny            10
Tucker           10
Lola             10
Winston           9
Bo                9
the               8
Sadie             8
Buddy             7
Bailey            7
Daisy             7
Toby              7
an                7
Oscar             6
Scout             6
Dave              6
Leo               6
Koda              6
Jax               6
Bella             6
Stanley           6
Jack              6
Rusty             6
Milo              6
Phil              5
               ... 
Tessa             1
Geno              1
Rooney            1
Shiloh            1
Jockson           1
Scott             1
Fabio             1
Tycho             1
Walker            1
Baron             1
Barry             1
Boston            1
Sweets            1
Amber             1
unacceptable      1
Tommy             1
Barney            1
Monkey            1
Dallas            1


In [18]:
# Checking denominator values
csv_df[csv_df['rating_denominator'] != 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
342,832088576586297345,8.320875e+17,3.058208e+07,2017-02-16 04:45:50 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@docmisterio account started on 11/15/15,NaN,NaN,NaN,NaN,11,15,None,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/820690176645140481/photo/1,https://twitter.com/dog_rates/status/820690176645140481/photo/1,https://twitter.com/dog_rates/status/820690176645140481/photo/1",84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://twitter.com/dog_rates/status/810984652412424192/photo/1",24,7,Sam,None,None,None,None
784,775096608509886464,NaN,NaN,2016-09-11 22:20:06 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",7.403732e+17,4.196984e+09,2016-06-08 02:41:38 +0000,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,None,None,None,None,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244762497024/video/1,165,150,None,None,None,None,None
1068,740373189193256964,NaN,NaN,2016-06-08 02:41:38 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1,https://twitter.com/dog_rates/status/740373189193256964/photo/1",9,11,None,None,None,None,None
1120,731156023742988288,NaN,NaN,2016-05-13 16:15:54 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,NaN,NaN,NaN,https://twitter.com/dog_rates/status/731156023742988288/photo/1,204,170,this,None,None,None,None
1165,722974582966214656,NaN,NaN,2016-04-21 02:25:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,NaN,NaN,NaN,https://twitter.com/dog_rates/status/722974582966214656/photo/1,4,20,None,None,None,None,None
1202,716439118184

In [19]:
# Checking numerator values higher than 14
csv_df[csv_df['rating_numerator'] > 14].shape[0]

28

### Assessing `predictions_df`
Sample view:

In [20]:
# View predictions tsv file 'twitter-archive-enhanced.csv' dataframe
predictions_df.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
918,701601587219795968,https://pbs.twimg.com/media/CbyW7B0W8AIX8kX.jpg,1,Chihuahua,0.993661,True,Pembroke,0.001505,True,toy_terrier,0.000867,True
951,704871453724954624,https://pbs.twimg.com/media/Ccg02LiWEAAJHw1.jpg,1,Norfolk_terrier,0.689504,True,soft-coated_wheaten_terrier,0.101480,True,Norwich_terrier,0.055779,True
1140,729823566028484608,https://pbs.twimg.com/media/CiDap8fWEAAC4iW.jpg,1,kelpie,0.218408,True,Arabian_camel,0.114368,False,coyote,0.096409,False
1354,760252756032651264,https://pbs.twimg.com/media/Coz12OLWgAADdys.jpg,1,radio_telescope,0.155279,False,dam,0.154515,False,crane,0.098040,False
1210,742465774154047488,https://pbs.twimg.com/media/Ck3EribXEAAPhZn.jpg,1,web_site,0.997154,False,comic_book,0.000439,False,desktop_computer,0.000268,False


Check for missing values

In [21]:
# Check null values
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [22]:
# Checking for duplicated tweet ids
predictions_df[predictions_df['tweet_id'].duplicated()]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


### Assessing `json_df`
Sample view:

In [23]:
# View predictions tsv file 'tweet_json.txt' dataframe
json_df.sample(5)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
1372,NaN,NaN,2016-02-17 02:54:04,"[0, 94]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 699788867692531712, 'id_str': '699788867692531712', 'indices': [71, 94], 'media_url': 'http://pbs.twimg.com/media/CbYmRHyWEAASNzm.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CbYmRHyWEAASNzm.jpg', 'url': 'https://t.co/5bjrIRqByp', 'display_url': 'pic.twitter.com/5bjrIRqByp', 'expanded_url': 'https://twitter.com/dog_rates/status/699788877217865730/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 750, 'h': 1000, 'resize': 'fit'}, 'small': {'w': 510, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 750, 'h': 1000, 'resize': 'fit'}}}]}","{'media': [{'id': 699788867692531712, 'id_str': '699788867692531712', 'indices': [71, 94], 'media_url': 'http://pbs.twimg.com/media/CbYmRHyWEAASNzm.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CbYmRHyWEAASNzm.jpg', 'url': 'https://t.co/5bjrIRqByp', 'display_url': 'pic.twitter.com/5bjrIRqByp', 'expanded_url': 'https://twitter.com/dog_rates/status/699788877217865730/photo/1', 'type': 'photo', 'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'}, 'large': {'w': 750, 'h': 1000, 'resize': 'fit'}, 'small': {'w': 510, 'h': 680, 'resize': 'fit'}, 'medium': {'w': 750, 'h': 1000, 'resize': 'fit'}}}]}",2196,False,Say hello to Nala. She's a Freckled High Bruschetta. Petable af. 12/10 https://t.co/5bjrIRqByp,NaN,...,NaN,NaN,NaN,NaN,500,False,NaN,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",False,"{'id': 4196983835, 'id_str': '4196983835', 'name': 'WeRateDogs®', 'screen_name': 'dog_rates', 'location': '「 DM YOUR DOGS 」', 'description': 'Your Only Source For Professional Dog Ratings Instagram and Facebook ➪ WeRateDogs partnerships@weratedogs.com ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀', 'url': 'https://t.co/Wrvtpnv7JV', 'entities': {'url': {'urls': [{'url': 'https://t.co/Wrvtpnv7JV', 'expanded_url': 'https://blacklivesmatters.carrd.co', 'display_url': 'blacklivesmatters.carrd.co', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 8843137, 'friends_count': 17, 'listed_count': 5767, 'created_at': 'Sun Nov 15 21:41:29 +0000 2015', 'favourites_count': 145873, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 12662, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '000000', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1267972589722296320/XBr04M6J_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1267972589722296320/XBr04M6J_normal.jpg', 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/4196983835/1591077312', 'profile_link_color': 'F5ABB5', 'profile_sidebar_border_color': '000000', 'profile_sidebar_fill_color': '000000', 'profile_text_color': '000000', 'profile_use_background_image': False, 'has_extended_profile': False, 'default_profile': False, 'default_profile_image': False, 'following': False, 'follow_request_sent': False, 'notifications': False, 'translator_type': 'none'}"
1252,NaN,NaN,2016-03-13 23:24:56,"[0, 138]","{'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [], 'media': [{'id': 709158324613324805, 'id_str': '709158324613324805', 'indices': [115, 138], 'media_url': 'http://pbs.twimg.com/media/CddvvSwWoAUObQw.jpg', 'media_url_https': 'https://pbs.twimg.com/media/CddvvSwWoAUObQw.jpg', 'url': 'https://t.co

In [24]:
# A lot of columns, I need a better view
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 32 columns):
contributors                     0 non-null float64
coordinates                      0 non-null float64
created_at                       2331 non-null datetime64[ns]
display_text_range               2331 non-null object
entities                         2331 non-null object
extended_entities                2059 non-null object
favorite_count                   2331 non-null int64
favorited                        2331 non-null bool
full_text                        2331 non-null object
geo                              0 non-null float64
id                               2331 non-null int64
id_str                           2331 non-null int64
in_reply_to_screen_name          77 non-null object
in_reply_to_status_id            77 non-null float64
in_reply_to_status_id_str        77 non-null float64
in_reply_to_user_id              77 non-null float64
in_reply_to_user_id_str          77 n

In [25]:
# Checking for duplicated tweet ids
json_df[json_df['id'].duplicated()]

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user


Now that I have assesed all dataframes I will enlist all my findings:

### Quality Issues

- `csv_df` 
 - Some values in name column are not names
 - Denominators different than 10 (0 for example)
 - Some numerators are too high to be correct (1776 for example)
 - There are 220 tweets with no photo uploaded since they used a video instead
 - 78 records are replies
 - 181 records are retweets
 - Some records have more than one stage of dog
 - Some tweets do not include dogs
- `predictions_df`
 - Images are of different size


### Tidiness Issues

- `csv_df`
 - Source column is not necessary for this analysis
 - Reply and retweet info not necessary after cleaning replies and retweets
 - Column "expanded_urls" not necessary after removing tweets without photo
 - Stage of dog is in separate columns (doggo, floofer, pupper and puppo)
 
- `predictions_df`
 - For these columns I only need one column with the dog prediction that has the highest confidence:    img_num p1	p1_conf	p1_dog	p2	p2_conf	p2_dog	p3	p3_conf	p3_dog
 
- `json_df`
 - Most of the columns do not have interesting information for this analysis

## Clean
Now that we have identified quality and tidiness issues, it is time to clean, but first we need to create new copies of the dataframes:

In [26]:
# Create copies for each dataframe
csv_df_clean = csv_df.copy()
predictions_df_clean = predictions_df.copy()
json_df_clean = json_df.copy()

Ok, copies are ready, now to start working issue by issue using the define, code and test method:
###  Quality `csv_df`- Some values in name column are not names
#### Define
Replace None and the most common values (>1 ocurrence) misintrepreted as names ('a','the','an','one' ,'just', 'getting') as NaN
#### Code

In [27]:
# List of values to replace
to_replace = ['None','a','the','an','one' ,'just', 'getting']

# Replace in clean df
csv_df_clean.name.replace(to_replace,'', inplace = True)

#### Test

In [28]:
# Check if replaced values are still present in data
csv_df_clean[csv_df_clean['name'].isin(to_replace)]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `csv_df` - Denominators different than 10 (0 for example)
#### Define
Remove 23 records with denominators different than 10 (most of them are because they are rating groups of dogs)

#### Code

In [29]:
# Reassigning without removed rows
csv_df_clean = csv_df_clean[csv_df_clean['rating_denominator'] == 10]

#### Test

In [30]:
# Check for rows with ratings different than 10
csv_df_clean[csv_df_clean['rating_denominator'] != 10]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `csv_df` - Some numerators are too high to be correct (1776 for example)
#### Define
Remove 28 records with numerators higher than 14 (14 is the valid current highest record)

#### Code

In [31]:
# Reassigning without removed rows
csv_df_clean = csv_df_clean[csv_df_clean['rating_numerator'] <= 14]

#### Test

In [32]:
# Check for rows with ratings higher than 14
csv_df_clean[csv_df_clean['rating_numerator'] > 14]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `csv_df` - There are 220 tweets with no photo uploaded since they used a video instead
#### Define
Remove 220 records without photos

#### Code

In [33]:
# Reassigning with only tweets that contain photo(s)
csv_df_clean = csv_df_clean[csv_df_clean.expanded_urls.str.contains('photo', na=False)]

#### Test

In [34]:
# Checking tweets without photo
csv_df_clean[csv_df_clean.expanded_urls.str.contains('photo')==False]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `csv_df` -  78 records are replies
#### Define
Remove 78 records that are replies

#### Code

In [35]:
# Reassigning without records that are replies
csv_df_clean = csv_df_clean[csv_df_clean.in_reply_to_status_id.isna()]

#### Test

In [36]:
# Checking for removed rows
csv_df_clean[~csv_df_clean.in_reply_to_status_id.isna()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `csv_df` -  181 records are retweets
#### Define
Remove 181 records that are retweets

#### Code

In [37]:
# Reassigning without records that are retweets
csv_df_clean = csv_df_clean[csv_df_clean.retweeted_status_id.isna()]

#### Test

In [38]:
# Checking for removed rows
csv_df_clean[~csv_df_clean.retweeted_status_id.isna()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `csv_df` -  Some records have more than one stage of dog
#### Define
Remove 14 records that have more than one stage of dog (doggo, pupper, etc) since they are not 100% defined.

#### Code

In [39]:
# Remove records with more than one dog stage by counting the number of None values in each row and assign to updated df
csv_df_clean = csv_df_clean[(csv_df_clean.iloc[:,13:17] == "None").sum(axis=1) > 2]

Test

In [40]:
# Checking for rows with more than one dog stage
csv_df_clean[(csv_df_clean.iloc[:,13:17] == "None").sum(axis=1) < 3]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


### Quality `predictions_df` - Images are of different size
#### Define
Add ":thumb" to the end of each image link `jpg_url` to ensure they are of the same size
#### Code

In [41]:
# Add string to each value in jpg_url
predictions_df_clean['jpg_url'] = predictions_df_clean['jpg_url'] + ':thumb'

#### Test

In [42]:
# Check some values to confirm change
predictions_df_clean.jpg_url.sample(3)

389    https://pbs.twimg.com/media/CVhBLohWEAAXtYl.jpg:thumb
382    https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg:thumb
581    https://pbs.twimg.com/media/CWt-MNIWEAAUC9S.jpg:thumb
Name: jpg_url, dtype: object

I have one more Quality Issue but first I need to join with other df to confirm if the tweet contains a dog photo, so let's start solving tidiness issues
### Tidiness `csv_df` - Stage of dog is in separate columns (doggo, floofer, pupper and puppo)
#### Define
Add one column that contains the stage of the dog
#### Code

In [43]:
# Create extra column with Dog Stage by concatenating 4 different columns (I could not find a better solution)
csv_df_clean['dog_stage'] = csv_df_clean['doggo'] + csv_df_clean['floofer'] +  csv_df_clean['pupper'] + csv_df_clean['puppo']

# Remove extra Nones and change case
csv_df_clean['dog_stage'] = csv_df_clean.dog_stage.str.replace('None','').str.title()

#### Test

In [44]:
# Confirm values of new column
csv_df_clean.dog_stage.value_counts()

           1595
Pupper      191
Doggo        55
Puppo        22
Floofer       6
Name: dog_stage, dtype: int64

### Tidiness `csv_df` - Removing unnecesary columns:
- Source column is not necessary for this analysis
- Reply and retweet info not necessary after cleaning replies and retweets
- Column "expanded_urls" not necessary after removing tweets without photo 
- Stage of dog extra columns (doggo, floofer, pupper and puppo)

#### Define
Remove columns no longer needed (`in_reply_to_status_id`, `in_reply_to_user_id`, `source`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, `expanded_urls`,`doggo`, `floofer`, `pupper`, `puppo`)

#### Code

In [45]:
# Keeping only needed columns
csv_df_clean = csv_df_clean[['tweet_id','timestamp','text','rating_numerator',
       'rating_denominator', 'name', 'dog_stage']]

#### Test

In [46]:
# Check new column set
csv_df_clean.columns

Index(['tweet_id', 'timestamp', 'text', 'rating_numerator',
       'rating_denominator', 'name', 'dog_stage'],
      dtype='object')

### Tidiness `predictions_df` - For these columns I only need one column with the dog prediction that has the highest confidence: img_num p1 p1_conf p1_dog p2 p2_conf p2_dog p3 p3_conf p3_dog
#### Define
Create new column with best dog prediction and if none are dogs set the value to "Not a dog"
#### Code

In [47]:
# Conditions to select the best dog prediction
# Default Value
predictions_df_clean['breed'] = predictions_df_clean['p1']

# When p1 is not a dog
predictions_df_clean.loc[predictions_df_clean['p1_dog'] == False,'breed'] = predictions_df_clean['p2']

# When p1 and p2 are not a dog
predictions_df_clean.loc[(predictions_df_clean['p1_dog'] == False)&
                         (predictions_df_clean['p2_dog'] == False),'breed'] = predictions_df_clean['p3']

# When p1 and p2 and p3 are not a dog
predictions_df_clean.loc[(predictions_df_clean['p1_dog'] == False)&
                         (predictions_df_clean['p2_dog'] == False)&
                         (predictions_df_clean['p3_dog'] == False),'breed'] = "Not a dog!"

In [48]:
# Some beautifying for the new colum
predictions_df_clean['breed'] = predictions_df_clean['breed'].str.replace('_', ' ').str.title()

In [49]:
# Drop unnecesary columns
predictions_df_clean = predictions_df_clean[['tweet_id', 'jpg_url','breed']]

#### Test

In [50]:
# Check new view
predictions_df_clean.sample(3)

,tweet_id,jpg_url,breed
402,673697980713705472,https://pbs.twimg.com/media/CVl0vFeWoAAMTfg.jpg:thumb,Not A Dog!
2061,889638837579907072,https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg:thumb,French Bulldog
1347,759447681597108224,https://pbs.twimg.com/media/CooZok_WEAA7oPw.jpg:thumb,Kuvasz


### Tidiness `json_df` - Most of the columns do not have interesting information for this analysis
#### Define
Keep only columns `favorite_count`, `id` and `retweet_count`
#### Code

In [51]:
# Keep only relevant columns
json_df_clean = json_df_clean[['id', 'favorite_count','retweet_count']]

# Rename id to tweet_id
json_df_clean = json_df_clean.rename(columns={'id':'tweet_id'})

#### Test

In [52]:
# Check new view
json_df_clean.sample(3)

,tweet_id,favorite_count,retweet_count
446,817502432452313088,0,3390
845,761976711479193600,5386,1994
611,794355576146903043,0,10354


We only have one more quality issue to solve which is remove non-dogs from the dataframe, but first, I am going to merge the 3 dataframes into one master since I need the prediction of the dog.

In [53]:
# Merging the 3 dataframes into 1 master 
twitter_archive_master_df = pd.merge(csv_df_clean,predictions_df_clean,on='tweet_id').merge(json_df_clean,on='tweet_id')

# Let's see our result
twitter_archive_master_df.sample(5)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dog_stage,jpg_url,breed,favorite_count,retweet_count
789,727685679342333952,2016-05-04 02:26:00 +0000,This is Cilantro. She's a Fellation Gadzooks. Eyes are super magical af. 12/10 could get lost in https://t.co/yJ26LNuyj5,12,10,Cilantro,,https://pbs.twimg.com/media/ChlCQg-VIAQ_8g4.jpg:thumb,Border Collie,2905,629
1790,667211855547486208,2015-11-19 05:24:37 +0000,This is Genevieve. She is a golden retriever cocktail mix. Comfortable close to wall. Shows no emotions. 9/10 https://t.co/azEoGqVonH,9,10,Genevieve,,https://pbs.twimg.com/media/CUJppKJWoAA75NP.jpg:thumb,Golden Retriever,460,223
495,780543529827336192,2016-09-26 23:04:13 +0000,Here's a perturbed super floof. 12/10 would snug so damn well https://t.co/VG095mi09Q,12,10,,,https://pbs.twimg.com/media/CtUMLzRXgAAbZK5.jpg:thumb,Golden Retriever,6318,1748
54,879376492567855104,2017-06-26 16:31:08 +0000,This is Jack AKA Stephen Furry. You're not scoring on him. Unless he slips down the slide. 12/10 would happily get blocked by https://t.co/0gOi601EAa,12,10,Jack,,https://pbs.twimg.com/media/DDQsQGFV0AAw6u9.jpg:thumb,Labrador Retriever,15625,2820
1586,671115716440031232,2015-11-29 23:57:10 +0000,"Meet Phred. He isn't steering, looking at the road, or wearing a seatbelt. Phred is a rolling tornado of danger 6/10 https://t.co/mZD7Bo7HfV",6,10,Phred,,https://pbs.twimg.com/media/CVBILUgVAAA1ZUr.jpg:thumb,Malinois,1298,730


### Quality `twitter_archive_master_df`  - Some tweets do not include dogs
#### Define
Remove Tweets were the predicted image was not of a dog 

#### Code

In [54]:
# Remove records with predicted breed as "Not A Dog!"
twitter_archive_master_df = twitter_archive_master_df[twitter_archive_master_df['breed'] != "Not A Dog!"]

#### Test

In [55]:
# Check if we still have no-dogs
twitter_archive_master_df[twitter_archive_master_df['breed'] == "Not A Dog!"]

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dog_stage,jpg_url,breed,favorite_count,retweet_count


## Saving cleaned data to a CSV
Now that we have cleaned and merged all of our data we are going to save it to a csv

In [56]:
# Saving dataframe to csv
twitter_archive_master_df.to_csv('twitter_archive_master.csv')

## Insights and visualizations from our data
For the final section of the notebook, let's get some insights and visualizations of our data **using the assesed and cleaned version of our initial sources** :)

### Top 10 rated Breeds

In [57]:
# Generating top 10 rate average by breed
twitter_archive_master_df.groupby('breed')['rating_numerator'].mean().nlargest(10)

breed
Saluki                12.500000
Briard                12.333333
Border Terrier        12.142857
Silky Terrier         12.000000
Standard Schnauzer    12.000000
Tibetan Mastiff       12.000000
Gordon Setter         11.750000
Samoyed               11.621622
Golden Retriever      11.517241
Australian Terrier    11.500000
Name: rating_numerator, dtype: float64

It seems like the page likes hairy dogs

### Top rated dog stage

In [58]:
# Generating top 10 rate averages by dog stage
twitter_archive_master_df.groupby('dog_stage')['rating_numerator'].mean().nlargest(4)

dog_stage
Floofer    12.000000
Puppo      11.952381
Doggo      11.829787
Pupper     10.828025
Name: rating_numerator, dtype: float64

And this also shows love for the fur

### Most favorited and retweeted posts

In [59]:
# Most favorites for a post
twitter_archive_master_df.sort_values('favorite_count',ascending=False).head(5)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dog_stage,jpg_url,breed,favorite_count,retweet_count
286,822872901745569793,2017-01-21 18:26:02 +0000,Here's a super supportive puppo participating in the Toronto #WomensMarch today. 13/10 https://t.co/nTz3FtorBc,13,10,,Puppo,https://pbs.twimg.com/media/C2tugXLXgAArJO4.jpg:thumb,Lakeland Terrier,131525,43096
101,866450705531457537,2017-05-22 00:28:40 +0000,"This is Jamesy. He gives a kiss to every other pupper he sees on his walk. 13/10 such passion, much tender https://t.co/wk7TfysWHr",13,10,Jamesy,Pupper,https://pbs.twimg.com/media/DAZAUfBXcAAG_Nn.jpg:thumb,French Bulldog,115210,32461
306,819004803107983360,2017-01-11 02:15:36 +0000,This is Bo. He was a very good First Doggo. 14/10 would be an absolute honor to pet https://t.co/AdPKrI8BZ1,14,10,Bo,Doggo,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg:thumb,Standard Poodle,87159,36894
86,870374049280663552,2017-06-01 20:18:38 +0000,This is Zoey. She really likes the planet. Would hate to see willful ignorance and the denial of fairly elemental science destroy it. 13/10 https://t.co/T1xlgaPujm,13,10,Zoey,,https://pbs.twimg.com/media/DBQwlFCXkAACSkI.jpg:thumb,Golden Retriever,77842,24223
368,806629075125202948,2016-12-07 22:38:52 +0000,"""Good afternoon class today we're going to learn what makes a good boy so good"" 13/10 https://t.co/f1h2Fsalv9",13,10,,,https://pbs.twimg.com/media/CzG425nWgAAnP7P.jpg:thumb,Cocker Spaniel,75263,34867


In [60]:
# Most retweets for a post
twitter_archive_master_df.sort_values('retweet_count',ascending=False).head(5)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dog_stage,jpg_url,breed,favorite_count,retweet_count
286,822872901745569793,2017-01-21 18:26:02 +0000,Here's a super supportive puppo participating in the Toronto #WomensMarch today. 13/10 https://t.co/nTz3FtorBc,13,10,,Puppo,https://pbs.twimg.com/media/C2tugXLXgAArJO4.jpg:thumb,Lakeland Terrier,131525,43096
306,819004803107983360,2017-01-11 02:15:36 +0000,This is Bo. He was a very good First Doggo. 14/10 would be an absolute honor to pet https://t.co/AdPKrI8BZ1,14,10,Bo,Doggo,https://pbs.twimg.com/media/C12whDoVEAALRxa.jpg:thumb,Standard Poodle,87159,36894
368,806629075125202948,2016-12-07 22:38:52 +0000,"""Good afternoon class today we're going to learn what makes a good boy so good"" 13/10 https://t.co/f1h2Fsalv9",13,10,,,https://pbs.twimg.com/media/CzG425nWgAAnP7P.jpg:thumb,Cocker Spaniel,75263,34867
101,866450705531457537,2017-05-22 00:28:40 +0000,"This is Jamesy. He gives a kiss to every other pupper he sees on his walk. 13/10 such passion, much tender https://t.co/wk7TfysWHr",13,10,Jamesy,Pupper,https://pbs.twimg.com/media/DAZAUfBXcAAG_Nn.jpg:thumb,French Bulldog,115210,32461
1364,676219687039057920,2015-12-14 01:58:31 +0000,This is Kenneth. He's stuck in a bubble. 10/10 hang in there Kenneth https://t.co/uQt37xlYMJ,10,10,Kenneth,,https://pbs.twimg.com/media/CWJqN9iWwAAg86R.jpg:thumb,Whippet,73935,29875


### Visualization
I made this visualization in Tableau where it shows the relation between average number of retweets and favorites for each breed:

In [61]:
%%html
<div class='tableauPlaceholder' id='viz1598670650489' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Do&#47;DogRateAnalysis&#47;DogRateAnalysis&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;DogRateAnalysis&#47;DogRateAnalysis?:language=en&amp;:embed=y&amp;:display_count=y&amp;publish=yes' /> <param name='toolbar' value='no' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Do&#47;DogRateAnalysis&#47;DogRateAnalysis&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1598670650489');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='900px';vizElement.style.height='600px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>